In [260]:
import requests
import json
import numpy as np
from sklearn.model_selection import train_test_split
import time

In [226]:
WIT_API_HOST = 'https://api.wit.ai'
WIT_API_VERSION = '20200513'

In [227]:
with open('token.txt') as file:
    TOKEN = file.read()
    
HEADERS = {
    'authorization': 'Bearer ' + TOKEN,
    'accept': 'application/vnd.wit.' + WIT_API_VERSION + '+json'
}

In [228]:
def getQueries():
    rsp = requests.get(WIT_API_HOST + '/utterances', headers=HEADERS, params={'limit': 1000})
    return rsp.json()

In [229]:
def delQueries(queries):
    if not queries:
        print('empty list, aborting')
        return
    confirm = input('CONFIRM DELETION: ')
    if confirm != 'yes':
        print('aborting delete')
        return
    print('deleting the following. \n')
    print(queries)
    
    queriesDelete = [{k: v for k, v in query.items() if k.startswith('text')} for query in queries]
    rsp = requests.delete(WIT_API_HOST + '/utterances', headers=HEADERS, data=str(queriesDelete))
    
    print(rsp.json())

In [248]:
# [a,b,c]
def getExamples(filename):
    examples = np.loadtxt(filename, dtype='str', delimiter='\n')
    return examples

In [249]:
# [[a,x],[b,x],[c,x]]
def labelExamples(examples, label):
    reshaped = np.reshape(examples, (-1, 1))
    labels = np.full(examples.shape[0], label).reshape(-1, 1)
    return np.concatenate((reshaped, labels), 1)

In [250]:
# [ task1 [[a,x],[b,x],[c,x]] , task2 [[d,y],[e,y],[f,y]] ]
def getLabeledExamples(filenames, labels):
    data = []
    for i in range(0, len(filenames)):
        labeled = labelExamples(getExamples(filenames[i]), labels[i])
        data.append(labeled)
    return data

In [251]:
# [ train: task1 + task2 , test: task1 + task2 ]
def formTrainTest(data):
    trains = []
    tests = []
    for dataset in data:
        a, b = train_test_split(dataset, test_size=0.5)
        trains.append(a)
        tests.append(b)
        
    train = np.empty((0,2))
    test = np.empty((0,2))
    for i in range(len(trains)):
        train = np.concatenate((train, trains[i]))
        test = np.concatenate((test, tests[i]))
        
    return train, test

In [252]:
def examplesToJson(examples):
    json = []
    for example in examples:
        json.append({'text': example[0], 'intent': example[1], 'entities': [], 'traits': []})
    return json

In [262]:
data = getLabeledExamples(['class.txt', 'reg.txt'], ['task_class', 'task_reg'])
train, test = formTrainTest(data)
trainJson = examplesToJson(train)
testJson = examplesToJson(test)

In [263]:
#str(testJson)

In [264]:
rsp = requests.post(WIT_API_HOST + '/utterances', headers=HEADERS, data=str(trainJson))
rsp.json()

{'sent': True, 'n': 40}

In [265]:
time.sleep(30) # DELAY - ALLOW IT TO TRAIN

In [266]:
for testExample in testJson:
    rsp = requests.get(WIT_API_HOST + '/message', headers=HEADERS, params={'q': testExample['text']})
    rspJson = rsp.json()
    print(rspJson)
    
    print(len(rspJson['intents']) > 0 and rspJson['intents'][0]['name'] == testExample['intent'])
    print()

{'text': 'classifier to identify high-risk populations', 'intents': [{'id': '1728532153951407', 'name': 'task_class', 'confidence': 0.9419}], 'entities': {}, 'traits': {}}
True

{'text': 'automated detection of safe drinking water', 'intents': [{'id': '184248982902938', 'name': 'task_reg', 'confidence': 0.8001}], 'entities': {}, 'traits': {}}
False

{'text': 'classifies spam texts', 'intents': [{'id': '1728532153951407', 'name': 'task_class', 'confidence': 0.936}], 'entities': {}, 'traits': {}}
True

{'text': 'decide if an email was spoofed', 'intents': [{'id': '1728532153951407', 'name': 'task_class', 'confidence': 0.9287}], 'entities': {}, 'traits': {}}
True

{'text': 'determines the level of security of a codebase', 'intents': [{'id': '1728532153951407', 'name': 'task_class', 'confidence': 0.8202}], 'entities': {}, 'traits': {}}
True

{'text': 'image classification on cifar-10', 'intents': [{'id': '1728532153951407', 'name': 'task_class', 'confidence': 0.5654}], 'entities': {}, 'tra

In [247]:
getQueries()

[]

In [261]:
delQueries(getQueries())

CONFIRM DELETION:  yes


deleting the following. 

[{'text': 'phishing detection', 'entities': [], 'traits': [], 'intent': {'id': '1728532153951407', 'name': 'task_class'}}, {'text': 'project the quantity of drinks', 'entities': [], 'traits': [], 'intent': {'id': '184248982902938', 'name': 'task_reg'}}, {'text': 'project median incomes for 2023', 'entities': [], 'traits': [], 'intent': {'id': '184248982902938', 'name': 'task_reg'}}, {'text': 'classifies spam texts', 'entities': [], 'traits': [], 'intent': {'id': '1728532153951407', 'name': 'task_class'}}, {'text': 'forecast the weather', 'entities': [], 'traits': [], 'intent': {'id': '184248982902938', 'name': 'task_reg'}}, {'text': 'predict whether trump will get re-elected', 'entities': [], 'traits': [], 'intent': {'id': '1728532153951407', 'name': 'task_class'}}, {'text': 'detect if a patient has bronchitis from xrays', 'entities': [], 'traits': [], 'intent': {'id': '1728532153951407', 'name': 'task_class'}}, {'text': 'automated detection of safe drinking w